In [43]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import numpy as np
import math
import nltk
import nltk
nltk.download('punkt')

nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [44]:
df = pd.read_csv("spam.csv",encoding="Windows-1251")
# df = pd.read_csv("/content/spam.csv")
X = df['v2']
y = np.where(df['v1']=='ham',1,0)


In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

stop_words = set(stopwords.words('english'))
preprocess = lambda text: ' '.join([word.lower() for word in word_tokenize(text) if word.isalpha() and word.lower() not in stop_words])
X_train = X_train.apply(preprocess)
X_test = X_test.apply(preprocess)


def calculate_tf(word, sentence):
    words = sentence.split()
    return words.count(word) / (len(words)+1)

def calculate_idf(word, sentences):
        no = 0
        for sent in sentences:
            if word in sent:
                no += 1
        return math.log(len(sentences) / (no + 1))



all_documents = X_train.tolist() + X_test.tolist()
idf_values = {}
unique_terms = set(' '.join(all_documents).split())

for term in unique_terms:
    idf_values[term] = calculate_idf(term, all_documents)


vocabulary = sorted(list(idf_values.keys()))

X_train_tfidf_manual = []

for document in X_train:
    tfidf_vector = []
    for term in vocabulary:
        tfidf_value = calculate_tf(term, document) * idf_values[term]
        tfidf_vector.append(tfidf_value)
    X_train_tfidf_manual.append(tfidf_vector)

X_test_tfidf_manual = []
for document in X_test:
    tfidf_vector = []
    for term in vocabulary:
        tfidf_value = calculate_tf(term, document) * idf_values[term]
        tfidf_vector.append(tfidf_value)
    X_test_tfidf_manual.append(tfidf_vector)

X_train_tfidf_manual = np.array(X_train_tfidf_manual)
X_test_tfidf_manual = np.array(X_test_tfidf_manual)


In [45]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train_tfidf_manual, y_train)


RandomForestClassifier()

In [46]:
new_text = "I HAVE A DATE ON SUNDAY WITH WILL!!"

# Calculate TF-IDF manually
new_text_processed = preprocess(new_text)
new_text_tfidf_manual = []
for term in vocabulary:
    tf = calculate_tf(term, new_text_processed)
    idf = idf_values[term]
    tf_idf = tf * idf
    new_text_tfidf_manual.append(tf_idf)

# Predict sentiment using the model
predicted_sentiment = model.predict([new_text_tfidf_manual])

# Print the predicted sentiment
if predicted_sentiment[0] == 1:
    print("Predicted Sentiment: Not Spam")
else:
    print("Predicted Sentiment: Spam")

Predicted Sentiment: Not Spam


In [47]:
new_text="Money Money free Money"
new_text_tfidf_manual = [calculate_tf(term, preprocess(new_text)) * idf_values[term] for term in vocabulary]
predicted_sentiment = model.predict([new_text_tfidf_manual])
print("Predicted Sentiment:", "Not Spam" if predicted_sentiment[0] == 1 else "Spam")

Predicted Sentiment: Not Spam
